#  FullStory Data Science Challenge

## Steven Yeh

### Challenge: <br>
> Imagine that you decide to drive a taxi for **10 hours each week** to earn a little extra money.
> Explain how you would approach **maximizing your income** as a taxi driver.
> If you could enrich the dataset, what would you add?  Is there anything in the dataset that you don’t find especially useful?

In addition to the resources provided, I also used the NYC Taxi & Limousine Commission website for further reference.
https://www1.nyc.gov/site/tlc/passengers/taxi-fare.page

In [1]:
# Python3.6

import pandas as pd
import os
import sys
import numpy as np
import sklearn
import time
import datetime
import matplotlib

#import warnings
#warnings.filterwarnings('ignore')

%matplotlib inline

pd.set_option('display.max_columns', 30)

Let's import the Yellow Taxi trip data from June 2017.

In [3]:
start = time.time()
path = 'path_to_csv'

raw = pd.read_csv(path + 'yellow_tripdata_2017-06.csv')

print(time.time() - start)

31.238171815872192


Looking at the columns, and the data dictionary, we can see which columns will and won't be useful to our analysis. We can also figure out if we need to create any more useful columns from the ones currently available. <br>

If I am maximing my income as a taxi driver, I should be interested in **total_amount**, which is the sum of **fare_amount**, **extra**, **mta_tax**, **tip_amount**, **tolls_amount**, and **improvement_surcharge**. However, I need to keep in mind that I only have 10 hours a week. I am going to make the assumption that total amount is largely driven by fare amount.

In [4]:
# Make a copy of the dataset
dat = raw.copy()
print(dat.shape)
print(dat.columns)
dat.head()

(9656993, 17)
Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount'],
      dtype='object')


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2017-06-08 07:52:31,2017-06-08 08:01:32,6,1.03,1,N,161,140,1,7.5,1.0,0.5,1.86,0.0,0.3,11.16
1,2,2017-06-08 08:08:18,2017-06-08 08:14:00,6,1.03,1,N,162,233,1,6.0,1.0,0.5,2.34,0.0,0.3,10.14
2,2,2017-06-08 08:16:49,2017-06-08 15:43:22,6,5.63,1,N,137,41,2,21.5,1.0,0.5,0.00,0.0,0.3,23.30
3,2,2017-06-29 15:52:35,2017-06-29 16:03:27,6,1.43,1,N,142,48,1,8.5,1.0,0.5,0.88,0.0,0.3,11.18
4,1,2017-06-01 00:00:00,2017-06-01 00:03:43,1,0.60,1,N,140,141,1,4.5,0.5,0.5,2.00,0.0,0.3,7.80


## 1. Useful and Not-so-Useful Columns

Since NYC taxis don't charge for extra passengers, the **passenger_count** column will not factor into **total_amount**. The **VendorID** and the **store_and_fwd_flag** fields are related to the TPEP record provider and don't seem relevant to my goal to maximize my income.

In [5]:
print(dat['store_and_fwd_flag'].value_counts())
print()
print(dat['VendorID'].value_counts())

N    9624906
Y      32087
Name: store_and_fwd_flag, dtype: int64

2    5281999
1    4374994
Name: VendorID, dtype: int64


There's over 250 different LocationIDs in the dataset, but the most common **PULocationID** to **DOLocationID** pairing by far (109,686 trips) is from ID 264 to ID 264, which are "unknown". In addition, NYC taxis charge a standard fare for most if not all trips that begin and end in Manhattan. Seeing the next 19 most common pairings start and end in Manhattan, the **total_amounts** for these trips will only be affected by **trip_distance** and the various **fare-related** charges.

In [6]:
info = pd.read_csv(path + 'taxi+_zone_lookup.csv')
print(info.shape)
#print(info.columns)

# Count the pairings
loc_id_ct = dat.groupby(['PULocationID', 'DOLocationID']).size().reset_index(name='Freq')

# Join with the Taxi zone lookup to get Boroughs and Zones
loc_id_ct = loc_id_ct.merge(info[['LocationID', 'Borough', 'Zone']], 
                left_on = 'PULocationID', right_on = 'LocationID').merge(info[['LocationID', 'Borough', 'Zone']], 
                                                                         left_on = 'DOLocationID', right_on = 'LocationID')
# Drop, rename, reorder columns
loc_id_ct.drop(['LocationID_x', 'LocationID_y'], axis=1, inplace=True)

loc_id_ct.rename(columns={'Borough_x': 'PUBorough', 'Borough_y': 'DOBorough', 'Zone_x': 'PUZone', 'Zone_y': 'DOZone'}, 
                 inplace=True)

loc_id_ct = loc_id_ct[['PULocationID', 'PUBorough', 'PUZone', 'DOLocationID', 'DOBorough', 'DOZone', 'Freq']]

# order by descending frequency
loc_id_ct.sort_values(by=['Freq'], ascending=False).head(20)

(265, 4)


,PULocationID,PUBorough,PUZone,DOLocationID,DOBorough,DOZone,Freq
2677,264,Unknown,NV,264,Unknown,NV,109686
21530,237,Manhattan,Upper East Side South,236,Manhattan,Upper East Side North,52701
21669,236,Manhattan,Upper East Side North,237,Manhattan,Upper East Side South,44254
21529,236,Manhattan,Upper East Side North,236,Manhattan,Upper East Side North,43586
21670,237,Manhattan,Upper East Side South,237,Manhattan,Upper East Side South,40820
14872,239,Manhattan,Upper West Side South,142,Manhattan,Lincoln Square East,25281
21803,239,Manhattan,Upper West Side South,238,Manhattan,Upper West Side North,24995
1542,237,Manhattan,Upper East Side South,162,Manhattan,Midtown East,24901
16516,237,Manhattan,Upper East Side South,161,Manhattan,Midtown Center,24233
2192,142,Manhattan,Lincoln Square East,239,Manhattan,Upper West Side South,23145


There is no charge to using cash or credit to pay for a taxi, so the **payment_type** column is not going to be useful here either except as a filter (cancelled trips, etc.)

More than 99% of the **improvement_surcharge** column contains only \$0.30 surcharges and is baked into the **total_amount**. The same can be said about the \$0.50 **mta_tax**. <br> 

We'll keep the **tolls_amount**, **extra** (mostly either 0, \$0.50, or \$1.00), and the **tip_amount** columns. About 94.6% of **tolls_amount** are 0, with another 4.9% being the discounted \$5.76 E-ZPass charge. I would treat both the **tolls_amount** and **extra** columns as categorical variables. I'd use one-hot encoding to capture the rush hour and overnight charges, as well as the discounted and peak/off-peak E-Zpass charges. There are enough unique (3,695) **tip amounts** to treat this column as continuous.

In [136]:
print(dat['improvement_surcharge'].value_counts().head())
print()
print(dat['mta_tax'].value_counts().head())
print()
print(dat['tolls_amount'].value_counts().head())
print()
print(dat['extra'].value_counts().head())
print()
print('There are ' + str(dat['tip_amount'].nunique()) + ' unique tip amounts')

 0.3    9648899
-0.3       4708
 0.0       3364
 1.0         22
Name: improvement_surcharge, dtype: int64

 0.50    9605702
 0.00      45982
-0.50       4576
 0.85        655
 0.25         56
Name: mta_tax, dtype: int64

0.00     9140256
5.76      470008
10.50       8242
12.50       5661
2.64        5088
Name: tolls_amount, dtype: int64

 0.0    4958655
 0.5    3086020
 1.0    1568003
 4.5      41475
-0.5       1582
Name: extra, dtype: int64

There are 3695 unique tip amounts


**RatecodeID** is mostly (97%) standard rates, but other rate codes should factor into my decision as well. For example, JFK and Newark have their own pricing structures such as a \$52 base fare between Manhattan and JFK or a \$17.50 Newark surcharge.

In [11]:
# 1: Standard Rate
# 2: JFK
# 3: Newark
# 4: Nassau / Westchester
# 5: Negotiated Fare
# 6: Group ride 

print(dat['RatecodeID'].value_counts().head())

1    9374033
2     221857
5      33852
3      21490
4       5439
Name: RatecodeID, dtype: int64


## 2. Feature Creation

We can create a more useful **trip_duration** column with the **tpep_pickup_datetime** and **tpep_dropoff_datetime** columns.

In [7]:
from datetime import datetime

dat['tpep_pickup_datetime'] = pd.to_datetime(dat['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S', errors='ignore')
dat['tpep_dropoff_datetime'] = pd.to_datetime(dat['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S', errors='ignore')

# how long each trip took in minutes
dat['trip_duration'] = (dat['tpep_dropoff_datetime'] - dat['tpep_pickup_datetime']) / np.timedelta64(1, 'm')

Also from the timestamps, we can also create metrics detailing what **hour of the day** and what **day of the week** the trip started. 

In [8]:
dat['Hour_of_Day'] = dat['tpep_pickup_datetime'].dt.hour

# weekdays go from 0 - 7
# 0 is Sunday, 7 is Saturday
dat['Day_of_Week'] = dat['tpep_pickup_datetime'].dt.weekday

Here is a correlation matrix between the continuous variables. Most important observation is that **fare_amount** and **total_amount** are almost perfectly positively correlated. **trip_duration** and **trip_distance** are highly positively correlated (77.9%). <br>
The **dollars_per** metrics are positively correlated (70.6%) with **fare_amount** and **total_amount** because the former are derived from the latter. <br>
**tip_amount** is somewhat positively correlated with **trip_distance** (55.1%) and **trip_duration** (48.8%).

In [50]:
corr_mat = dat[['trip_distance', 'fare_amount', 'tip_amount', 'total_amount', 
                'trip_duration', 'dollars_per_min', 'dollars_per_hour']].corr()
corr_mat.style.background_gradient(cmap='coolwarm')

,trip_distance,fare_amount,tip_amount,total_amount,trip_duration,dollars_per_min,dollars_per_hour
trip_distance,1,0.0480289,0.550888,0.0593491,0.779223,-0.0130329,-0.0130329
fare_amount,0.0480289,1,0.0311768,0.999847,0.0433536,0.705801,0.705801
tip_amount,0.550888,0.0311768,1,0.046923,0.487744,0.0518609,0.0518609
total_amount,0.0593491,0.999847,0.046923,1,0.0528997,0.705911,0.705911
trip_duration,0.779223,0.0433536,0.487744,0.0528997,1,-0.0241779,-0.0241779
dollars_per_min,-0.0130329,0.705801,0.0518609,0.705911,-0.0241779,1,1
dollars_per_hour,-0.0130329,0.705801,0.0518609,0.705911,-0.0241779,1,1


Interestingly, **fare_amount** is not correlated with **trip_distance**, nor is it correlated with **trip_duration**. 
This is because fare amount can have variable cost based on how much time the taxi driver sits in slow traffic, even if the trip duration and distance are the same. This can be illustrated in an example: <br>

A 10 minute trip for 2 miles: <br>
* Example 1: Taxi driver goes almost 20mph for 6 minutes, and 1mph for 4 minutes = (2 miles at 0.50 dollars per 1/5 mile) + (4 minutes of slow traffic at 0.50 dollars per minute) = **7 dollars** <br>
* Example 2: Taxi driver goes almost 30mph for 4 minutes, and 1mph for 6 minutes = (2 miles at 0.50 dollars per 1/5 mile) + (6 minutes of slow traffic at 0.50 dollars per minute) = **8 dollars**

From the newly created **trip_duration** column, we can also create "value" metrics that tell us how much **fare_amount** we generate **per minute** or **per hour**.

In [9]:
# "value"
dat['dollars_per_min'] = dat['fare_amount'] / dat['trip_duration'] 
dat['dollars_per_hour'] = 60*dat['dollars_per_min']

## 3. Maximizing Income

To get more insightful statistics, we can filter the data for obvious outliers and bad data.

In [17]:
# Filter for these:
# trips with 0 duration
# trips over 10 hours long (I get 10hrs a week)
# where the total amount was $0 or less
# payment codes for voided / refunded / disputed / unknown trips
# mysterious RatecodeID 99


dat = dat.loc[(dat['trip_duration'] != 0) & (dat['total_amount'] > 0) & (dat['payment_type'] <= 2) &
              (dat['trip_duration'] <= 600) & (dat['RatecodeID'] != 99)]

print(dat.shape)

(9564334, 22)


### Fare Amount

First let's look at different slices of the data in relation to **fare_amount**.

Looking at **RatecodeID**, there are large outliers in **fare_amount**. Realistically, I would not rely on Negotiated Fares (5) because the income generated varies from driver / passenger. There is not enough data for Group rides (6) to glean any insights. Nassau / Westchester trips are also very low in observations, but the rates are clearly defined. 

Looking at 50th percentile, it appears that in 2017, trips to Newark (3) (68.50/trip) had a higher median base fare than JFK (2) trips. Logically, this makes sense as for Newark trips, the standard fare gets charged in addition to the Newark Surcharge, making a trip from Manhattan more expensive on average. Nassau / Westchester (4) trips have much higher variability, despite having almost as high of a fare as Newark trips. Standard rides (1) comprise almost all rides and have the lowest median fare (9.50). This table also highlights outliers that should be removed by RatecodeID group.

In [19]:
dat_rate = dat.groupby('RatecodeID')['fare_amount'].describe().reset_index()
dat_rate

,RatecodeID,count,mean,std,min,25%,50%,75%,max
0,1,9290284.0,12.051902,219.073592,0.0,6.5,9.5,14.00,630461.82
1,2,217647.0,51.999249,0.176604,2.5,52.0,52.0,52.00,52.00
2,3,20448.0,68.719982,16.733947,9.5,63.5,68.5,75.50,216.50
3,4,5211.0,70.580608,43.067283,0.0,42.5,63.0,91.00,589.50
4,5,30705.0,64.767154,56.207063,0.0,21.5,59.0,85.00,984.00
5,6,39.0,30.566923,144.073501,2.5,2.5,2.5,4.25,900.00


By median duration, trips to JFK (2) are 51.25 minutes, longer than trips to Newark and Nassau / Westchester. JFK trips to not seem like good value if my fares are lower and I'm driving longer distances compared to the other special rate trips. 

In [65]:
dat.groupby('RatecodeID')['trip_duration'].describe().reset_index()#[['RatecodeID', '50%']]

,RatecodeID,count,mean,std,min,25%,50%,75%,max
0,1,9290284.0,13.888182,10.950270,0.016667,6.650000,11.050000,17.883333,599.366667
1,2,217647.0,51.525519,23.748634,0.016667,37.133333,51.250000,65.533333,599.850000
2,3,20448.0,43.533123,21.594261,0.016667,30.316667,40.000000,53.950000,528.100000
3,4,5211.0,39.320329,22.787168,0.033333,24.691667,35.400000,50.091667,431.833333
4,5,30705.0,17.067892,27.736286,0.016667,0.150000,0.833333,28.883333,592.050000
5,6,39.0,2.529060,4.832589,0.033333,0.091667,0.216667,2.833333,20.116667


<p align="center">

Just from these two tables, we can somewhat approximate how much money I would make in 10 hours going by median **fare_amount** and median **trip_duration**.

600 / 11.05 = 54-55 trips around Manhattan <br>
600 / 51.25 minutes = 11-12 trips to JFK <br>
600 / 40 minutes = 15 trips to Newark <br>
600 / 35.4 minutes = 16-17 trips to Nassau / Westchester <br>


| Rate Type | # of Trips | Dollars per Trip | Total Fare |
|------|------|------|-----|
| Standard | 55 | 9.50 | 522.50 |
| JFK | 12 | 52 | 624 |
| Newark | 15 | 68.50 | 1,027.50 |
| Nassau/Westchester | 16 to 17 | 63 | 1,008 to 1,071 |

From this, I can't go wrong with either Newark or Nassau / Westchester trips

However, this assumes that I can survive 10 hour driving shifts. Also, this doesn't tell me when during the week I should be driving.

We can create a more realistic schedule by looking at the day of the week in addition to RatecodeID. It appears the Newark trips generate the best income over all days of the week compared to other rates. 

In [69]:
dat.groupby(['Day_of_Week', 'RatecodeID'])['fare_amount'].describe().reset_index().sort_values(by=['50%'], ascending=False)

,Day_of_Week,RatecodeID,count,mean,std,min,25%,50%,75%,max
20,3,3,3680.0,69.708614,16.968402,20.00,64.000,70.00,77.000,187.00
14,2,3,2781.0,69.460446,16.361894,20.00,63.500,69.50,76.500,180.00
26,4,3,4056.0,69.992604,17.373789,20.00,63.500,69.50,78.000,199.00
8,1,3,2492.0,69.212079,16.328481,20.00,63.500,69.50,76.125,161.50
2,0,3,2483.0,67.914217,16.611529,9.50,63.000,68.50,74.500,144.00
38,6,3,2936.0,67.073059,15.641910,17.00,63.500,67.00,72.000,216.50
32,5,3,2020.0,66.121287,17.174047,20.00,62.500,66.50,71.500,183.00
15,2,4,710.0,71.031620,47.333379,0.00,42.125,64.00,90.500,589.50
21,3,4,883.0,70.778256,40.822945,0.00,44.750,63.50,90.500,443.00
27,4,4,949.0,71.329557,43.155548,0.00,44.500,63.00,91.000,377.00


Now that we have established Newark trips as the best rate, we can decide what hour of what day of the week is optimal.

In [72]:
dat.loc[dat['RatecodeID'] == 3].groupby(['Day_of_Week', 'Hour_of_Day'])['fare_amount'].describe().reset_index().sort_values(by=['50%'], ascending=False)

,Day_of_Week,Hour_of_Day,count,mean,std,min,25%,50%,75%,max
112,4,16,192.0,80.679688,21.384849,20.0,76.000,84.25,91.500,127.0
89,3,17,189.0,79.293651,21.190618,20.0,71.000,80.50,90.500,137.0
111,4,15,312.0,78.376603,20.482798,20.0,72.875,80.00,88.000,170.0
113,4,17,171.0,75.944444,22.326155,20.0,72.000,80.00,86.000,160.0
65,2,17,140.0,77.196429,19.367210,20.0,71.875,79.75,87.500,127.0
88,3,16,251.0,78.247809,19.432429,20.0,73.500,79.50,86.750,127.0
64,2,16,180.0,77.588889,17.434006,20.0,72.000,78.00,84.125,180.0
110,4,14,433.0,76.953811,17.377187,20.0,70.500,77.00,83.000,199.0
66,2,18,109.0,77.119266,20.219730,20.0,72.000,77.00,85.000,170.5
41,1,17,135.0,75.477778,19.941132,20.0,70.750,77.00,82.000,132.0


Just grabbing the top 10 day/hour slots, I can maximize my income by making Newark trips during these times of the week:

* Mondays from 17:00 to 18:00 
* Tuesdays from 16:00 to 19:00 
* Wednesdays from 16:00 to 18:00 
* Thursdays from 14:00 to 18:00 

I'd rather have continuous hours since a Newark trip averages 40 minutes, so I could start earlier on Tuesdays at 15:00 for \\$1.00 less in median **fare_amount**.

Updated schedule:

* Tuesdays from 15:00 to 19:00 
* Wednesdays from 16:00 to 18:00 
* Thursdays from 14:00 to 18:00 

Using the same math from before, I can calculate my total income from 40 minutes a trip. I will use the hour in which a trip starts to determine the fare. If the trip extends into two different hours, I will allocate fares proportionally to how long the trip was in each hour. Also, I won't start a trip if my end time is after the hours I am alloted to drive.

| Day | Hour | Trip Start Time | Fare |
| ---- | ---- | ---- | ---- |
| Tues | 15:00 | 15:00| 76 |
| Tues | 15:00 | 15:40| half 76 + half 78 = 77 |
| Tues | 16:00 | 16:20| 78 |
| Tues | 17:00 | 17:00| 79.75 |
| Tues | 17:00 | 17:40| half 79.75 + half 77 = 78.375|
| Tues | 18:00 | 18:20| 77 |
| Wednesday | 16:00 | 16:00| 79.50 |
| Wednesday | 16:00 | 16:40| half 79.50 + half 80.50 = 80|
| Wednesday | 17:00 | 17:20| 80.50 |
| Thursday | 14:00 | 14:00| 77 |
| Thursday | 14:00 | 14:40| half 77 + half 80 = 78.50|
| Thursday | 15:00 | 15:20| 80 |
| Thursday | 16:00 | 16:00| 84.25 |
| Thursday | 16:00 | 16:40| half 84.25 + half 80 = 82.125|
| Thursday | 17:00 | 17:20| 80 |


__Total Fare: \\$1,188__

This schedule gives me more income than the previous table.


### Tips

What happens if we look at tips? As seen before, tips are somewhat positively correlated with trip duration and trip distance. However, tips will generally be a much smaller piece of the **total_amount** than the fare amount.


In [7]:
dat.groupby('RatecodeID')['tip_amount'].describe().reset_index()

,RatecodeID,count,mean,std,min,25%,50%,75%,max
0,1,9374033.0,1.705163,2.191113,-74.00,0.0,1.36,2.36,444.0
1,2,221857.0,7.381085,6.119590,-68.00,0.0,10.00,11.71,312.0
2,3,21490.0,10.758038,9.316374,0.00,0.0,12.42,17.25,329.8
3,4,5439.0,8.064870,10.007875,0.00,0.0,5.00,14.06,80.0
4,5,33852.0,7.183814,11.673877,-60.16,0.0,2.00,11.85,300.0
5,6,91.0,1.857582,9.950380,0.00,0.0,0.00,0.00,70.0
6,99,231.0,1.144199,3.688440,0.00,0.0,0.00,0.00,25.0


Seems like Newark trips have a higher median tip amount (\\$12.42) than other trips, so there is no reason to deviate from the above schedule. I can use a similar approach as above to allocate median tips per hour per day to each trip. 

### Other Charges

Toll amounts will vary by where the passenger is going/coming from. Tolls are mostly set charges and shouldn't be as big of an income factor to chase. There is also a `$`0.30 Improvement Surcharge, a `$`0.50 MTA tax, and a possible `$`0.50 or `$`1 rush hour and overnight charges. There is also a `$`17.50 Newark Surcharge.

## 4. Future Works

Since the goal is to maximize income, it would be useful to have a **idle time** metric detailing how long the taxi sits in busy traffic or drives under 12mph (defined by the Taxi website). With an additional idle time metric, we can better approximate total fare. It would also be useful to have specific **latitude** and **longitude** for the pickup and dropoff points. Even though we have the drive distance and duration, we can use the geographical coordinates to specify further where trips start and end.

There are also interesting questions in this dataset that can be answered with classification / regression models such as identifying the most important (variable importance) drivers of fare amounts, or predicting trip duration. 

## 5. Conclusion

I can maximize my weekly total fare amount, and therefore maximize my weekly income by driving to and from Newark airport according to this 10hr/week schedule:

* Tuesdays from 15:00 to 19:00 
* Wednesdays from 16:00 to 18:00 
* Thursdays from 14:00 to 18:00 